### NREL Solar Resource Data - lat/long file import

#### This program uses the Solar Resource Data API provided by NREL and returns various solar power related parameters for a given latitude and longitude.  The goal of the program is to take in a file of lat and long values and iterate through, pulling the values of interest and then writing them to a new file in a format usable for creation of maps and analytical purposes.  


##### **DATA RETURNED**  
Average Direct Normal Irradiance - measured surface perpendicular to direct solar rays (avg_dni)  
Average Global Horizontal Irradiance - measured surface horizontal to the ground (avg_ghi)  
Average Tilt at Latitude (avg_lat_tilt)  


##### **INFORMATION & RESEARCH**
https://developer.nrel.gov/docs/solar/solar-resource-v1/   
https://en.wikipedia.org/wiki/Solar_irradiance  
https://www.3tier.com/en/support/solar-online-tools/what-global-horizontal-irradiance-solar-prospecting/  
https://www.pveducation.org/pvcdrom/properties-of-sunlight/measurement-of-solar-radiation   


In [ ]:
import pandas as pd
import csv
import requests, json, csv

In [ ]:
# review list of lat longs to use for reading into API
with open('location_values.csv', newline='') as csvfile:
    loc_reader = csv.reader(csvfile)
    for loc in loc_reader:
        print(loc, end="\n")

In [ ]:
# bring lat / long csv into a dataframe
file_name = 'location_values.csv'
df = pd.read_csv(file_name) 
df 

### RETRIEVE MONTHLY VALUES AND SAVE TO A FILE FOR ANALYSIS

In [ ]:
# file name for saving monthly results
monthly_filename = input("filename for monthly data including .csv: ")

In [ ]:
# iterate through csv and query using lat / long to get average monthly DNI and save to a csv

try:
    # create URL using values in database
    lat_count = 0
    long_count = 0
    while lat_count < len(df):
        lat = df.at[lat_count,'lat']
        long = df.at[long_count,'long']
        API_ENDPOINT = 'https://developer.nrel.gov/api/solar/solar_resource/v1.json?api_key=aO3yoslbHz6Pg9vrVdVGjA47Jrv1TnjOfOhDJj0L&lat=%s&lon=%s' % (lat,long)
        full_url = API_ENDPOINT
        print()
        print('URL: ', full_url)    

        # send out the request for information
        response = requests.get(full_url)
        # check its status to see if all is well
        print('Made request, response status: ', response.status_code)
        # determine if results were returned or another error code
        if response.status_code == 200:
            # 200 is all good
            results = json.loads(response.text)
        elif response.status_code == 429:
            # 429 is request limit reached
            raise Exception 
        else: 
            # other exception noted on NREL is 422 for invalid parameter
            print()
            print("One of your parameters is not valid.")

        # append to the file
        information = results['outputs']
        print_file = open(monthly_filename, 'a')
        print_file.write(str(lat) + "," + str(long) + "\n")
        for key, value in information['avg_dni']['monthly'].items():
            print_file.write('%s , %s\n' % (key, value))
        print_file.write("\n")
        print_file.close()

        # add 1 to lat / long counters to move to next values for next iteration
        lat_count = lat_count + 1
        long_count = long_count + 1
    else:
        print()
        print("You've reached the end of the file.")
        
except Exception:
    print()
    # print exception if the hourly request limit imposed by NREL is reached
    print("You've reached your hourly limit of 1000 requests.")

### RETRIEVE YEARLY VALUES AND SAVE TO A FILE FORMATTED FOR GIS ANALYSIS

In [ ]:
# file name for saving yearly results
yearly_filename = input("filename for yearly data including .csv: ")

In [ ]:
# iterate through csv and query using lat / long to get average yearly DNI

try:
    # create URL using values in database
    lat_count = 0
    long_count = 0
    while lat_count < len(df):
        lat = df.at[lat_count,'lat']
        long = df.at[long_count,'long']
        API_ENDPOINT = 'https://developer.nrel.gov/api/solar/solar_resource/v1.json?api_key=aO3yoslbHz6Pg9vrVdVGjA47Jrv1TnjOfOhDJj0L&lat=%s&lon=%s' % (lat,long)
        full_url = API_ENDPOINT
        print()
        print('URL: ', full_url)    

        # send out the request for information
        response = requests.get(full_url)
        # check its status to see if all is well
        print('Made request, response status: ', response.status_code)
        # determine if results were returned or another error code
        if response.status_code == 200:
            # 200 is all good
            results = json.loads(response.text)
        elif response.status_code == 429:
            # 429 is request limit reached
            raise Exception 
        else: 
            # other exception noted on NREL is 422 for invalid parameter
            print()
            print("One of your parameters is not valid.")

        # append to the file
        print_file = open(yearly_filename, 'a')
        print_file.write(str(lat) + "\n")
        print_file.write(str(long) + "\n")
        print_file.write(str(results['outputs']['avg_dni']['annual'])) # result needs to be a string, not integer
        print_file.write("\n")
        print_file.close()
    
        # add 1 to lat / long counters to move to next values for next iteration
        lat_count = lat_count + 1
        long_count = long_count + 1
    else:
        print()
        print("You've reached the end of the file.")
        
except Exception:
    print()
    # print exception if the hourly request limit imposed by NREL is reached
    print("You've reached your hourly limit of 1000 requests.")

In [ ]:
# write the request results to a dataframe
result_file = yearly_filename
result_df = pd.read_csv(result_file, header=None) 
result_df 

In [ ]:
# rearrange values in file to better format and add header info
df_new = (pd.DataFrame(result_df.values.reshape(-1, 3), 
                    columns=['lat','long','avg_yrly_dni']))
df_new

In [ ]:
# merge original dataframe with city and state with resultant, reformatted dataframe
df_all = pd.merge(df, df_new, on=['lat','long'], how='left')
df_all

In [ ]:
# write dataframe to a new csv
final_file = input("name of reformatted request result file, include .csv: ")
df_all.to_csv(final_file, index=False)